In [34]:
#!pip install selenium
import re
import pandas as pd
from selenium import webdriver
import selenium.webdriver.chrome.service as service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from datetime import datetime
import time

from tqdm.notebook import tqdm
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets.widgets import *


#nltk para análisis del texto
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer

#para la red neuronal que obtiene palabras
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

driver = webdriver.Chrome(ChromeDriverManager("2.38").install())
urls = ['https://idm.presidencia.gov.co/prensa/discursos#k=']+['https://idm.presidencia.gov.co/prensa/discursos#k=#s='+ str(20*k+1) for k in range(1,27)]



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Driver [C:\Users\USER\.wdm\drivers\chromedriver\win32\2.38\chromedriver.exe] found in cache


In [35]:
def procesar(text):
    text=text.lower()
    text=re.sub("</?.*?>"," <> ",text)
    text=re.sub("(\\d|\\W)+"," ",text)
    return text

def direcciones_web(urls:list): #Extrae las direcciones web de los vínculos de los discursos
    direcciones = dict()
    print('Obteniendo los links de los discursos...')
    for direccion in tqdm(urls):
        driver.get(direccion)
        elementos = driver.find_elements(By.XPATH, '//div[@class="item link-item"]')
        time.sleep(3)
        elem_a =  elementos[0].find_elements_by_xpath("//a")
        for i in elem_a:
            try:
                if len(i.text)>0:
                    direcciones[i.text]=i.get_attribute("href")
            except:
                print('error en', direccion)
    return direcciones

def tabla_discursos(urls, direcciones):
    print('Creando tabla...')
    df = pd.DataFrame(columns = ['Clase','Fecha','Ubicacion','Titulo','Texto','Link'])
    for url in tqdm(urls):
        driver.get(url)
        ms = driver.find_elements(By.XPATH, '//div[@class="ms-webpart-zone ms-fullWidth"]')
        texto_pag = ms[0].text.split('\n')
        for i in range(0,len(texto_pag),3):
            try:
                clase, titulo, lugar =  texto_pag[i:i+3]
                fecha = datetime.strptime(lugar.replace(' ','').split(',')[-1],'%d/%m/%Y')
                ubicacion = lugar.replace(' ','').split(',')[0]
                dic= {'Clase':clase, 'Titulo': titulo, 'Fecha': fecha,
                      'Ubicacion' : ubicacion, 'Link':direcciones[titulo]}
                df = df.append(dic, ignore_index=True)
            except:
                pass
        time.sleep(1)
    return df


def discursos(df):
    print('Obteniendo los discursos...')
    for i in tqdm(df.index):
        url = df['Link'][i]
        driver.get(url)
        disc = driver.find_elements(By.XPATH, '//div[@class="col s12 infoPaginas"]')
        time.sleep(1.5)
        df['Texto'][i] = procesar(disc[0].text.replace('\n',' '))
        
def descargar(urls):
    direcciones = direcciones_web(urls)
    tabla = tabla_discursos(urls,direcciones)
    discursos(tabla)
    tabla.sort_values('Fecha').reset_index()
    return tabla

def cargar_tabla(ruta):
    df = pd.read_csv(ruta)
    return df


In [36]:
opcion = 0
while opcion!= '1' and opcion!='2':
    opcion = input('Si desea descargar los discursos, presione 1.\nSi desea cargar los datos ya descargados, presione 2.\n')
if opcion == '1':
    print('Eligió descargar los datos:\n')
    tabla = descargar(urls)
    tabla.to_csv('Discursos.csv')
elif opcion == '2':
    tabla = cargar_tabla('Discursos.csv')
    try:
        tabla = tabla.drop('Unnamed: 0', axis=1)  
    except:
        pass

Si desea descargar los discursos, presione 1.
Si desea cargar los datos ya descargados, presione 2.
2


# Red neuronal de tf-idf
https://www.freecodecamp.org/news/how-to-extract-keywords-from-text-with-tf-idf-and-pythons-scikit-learn-b2a0f3d7e667/

In [37]:
stop=set(nltk.corpus.stopwords.words("spanish"))
cv=CountVectorizer(max_df=0.85,stop_words=stop)


def vectorizar(tabla):
    docs= tabla['Texto'].tolist()
    #create a vocabulary of words, 
    #ignore words that appear in 85% of documents, 
    #eliminate stop words
    return cv.fit_transform(docs)

def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):   
    sorted_items = sorted_items[:topn]
    score_vals = []
    feature_vals = []
    for idx, score in sorted_items:
        fname = feature_names[idx]
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

def palabras_clave(tabla,n):
    word_count_vector = vectorizar(tabla)
    tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
    tfidf_transformer.fit(word_count_vector)
    docs_test=tabla['Texto'].tolist()
    feature_names=cv.get_feature_names()
    keys = []
    for i in tabla.index:
        doc=docs_test[i]
        tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))
        sorted_items=sort_coo(tf_idf_vector.tocoo())
        keywords=extract_topn_from_vector(feature_names,sorted_items,n)
        keys.append(keywords)
    return keys

def agrupar_dias(table):
    fechas = list(set(list(table['Fecha'])))
    fechas.sort()
    df = pd.DataFrame(columns= ['Fecha','Texto'])
    lista= []
    for i in fechas:
        data = table.loc[table['Fecha']==i]
        string = ' '
        for k in data.index:
            string= string + data['Texto'][k]
        lista.append(string)
    df['Fecha'] = fechas
    df['Texto'] = lista
    return df

def contar_cumm(n):
    cols = list(tabla_dias['Palabras clave'][n].keys())
    tabla_evolucion = pd.DataFrame(index = tabla_dias['Fecha'], columns = cols)
    for i in tabla_evolucion.index:
        data = tabla_dias.loc[tabla_dias['Fecha']== i]
        for l in tabla_evolucion.columns:
            k = list(data['Texto'])[0].count(l)
            tabla_evolucion[l][i] = k
    return tabla_evolucion

def contar_palabra(tabla_dias, palabra):
    tabla_evolucion = pd.DataFrame(index = tabla_dias['Fecha'], columns = [palabra])
    for i in tabla_evolucion.index:
        data = tabla_dias.loc[tabla_dias['Fecha']== i]
        tabla_evolucion[palabra][i] = list(data['Texto'])[0].count(palabra)
    return tabla_evolucion

tabla['Palabras clave'] = palabras_clave(tabla,5)

In [38]:
tabla['Palabras clave'][0]

{'corrupción': 0.412,
 'transparencia': 0.289,
 'compliance': 0.193,
 'gurría': 0.166,
 'world': 0.161}

In [39]:
tabla_dias = agrupar_dias(tabla)
tabla_dias['Palabras clave'] = palabras_clave(tabla_dias,5)
tabla_dias

,Fecha,Texto,Palabras clave
0,2020-08-14,bogotá de agosto de colombianos muy buenas ta...,"{'sospecha': 0.387, 'minas': 0.287, 'antiperso..."
1,2020-08-17,bogotá de agosto de muchísimas gracias doctor...,"{'brien': 0.238, 'dictadura': 0.187, 'capítulo..."
2,2020-08-18,bogotá de agosto de colombianos muy buenas t...,"{'personas': 0.201, 'cómo': 0.194, 'rutas': 0...."
3,2020-08-19,bogotá de agosto de colombianos muy buenas ta...,"{'habilidades': 0.273, 'tic': 0.236, 'programa..."
4,2020-08-20,bogotá de agosto de colombianos muy buenas t...,"{'álvaro': 0.18, 'salud': 0.138, 'compromiso':..."
...,...,...,...
194,2021-06-01,quibdó de junio de estamos reunidos acá en l...,"{'quibdó': 0.467, 'chocó': 0.297, 'jóvenes': 0..."
195,2021-06-03,neiva de junio de quiero expresarles que hem...,"{'bloqueos': 0.317, 'jóvenes': 0.168, 'huila':..."
196,2021-06-04,bogotá de junio de señoras ministras es un h...,"{'comunal': 0.589, 'acción': 0.234, 'ceferino'..."
197,2021-06-05,parque isla de salamanca de junio de buenas ...,"{'ambiente': 0.247, 'árboles': 0.195, 'salaman..."


In [40]:
def update_plot(palabra_clave): #Crear un plot interactivo con la ecuación diferencial, igual  que en la celda anterior
    d = contar_cumm(palabra_clave)
    fig= px.line()
    for i in d.columns:
        fig.add_scatter(x=d.index, y=np.cumsum(d[i]), mode='lines', name = i)
    fig.update_layout(
                title={
                'text' : "Las palabras claves de la fecha <b> "+ str(tabla_dias['Fecha'][palabra_clave]) +"</b> son: " + str(', '.join(list(tabla_dias['Palabras clave'][palabra_clave].keys()))),
                'x':0.5,
                'xanchor': 'center',
                'font':{'size':15}
            },width  = 800,height =500) #Actualiza el entorno de la gráfica para modificar el título, la ubicación y su tamaño
    fig.show()
palabra_clave= widgets.FloatSlider(value=0, min=0,max=len(tabla_dias)-1,step=1, description= 'Palabras: ') #crear el slider de
widgets.interactive(update_plot,palabra_clave=palabra_clave) #Comando para actualizar la gráfica a medida que se mueve el slider

interactive(children=(FloatSlider(value=0.0, description='Palabras: ', max=198.0, step=1.0), Output()), _dom_c…

In [41]:
lista = []
for i in tabla['Palabras clave']:
    for j in i.keys():
        if j not in lista:
            lista.append(j)
lista.sort()
tags = nltk.pos_tag(lista)
dict_tags = {'CC': 'Conjunción de coordinación', 'CD' : 'número cardinal', 'DT' : 'Determinador', 'EX' : 'Existencial (allí)',
          'FW' : 'Palabra extranjera', 'EN' : 'Preposición o conjunción subordinada', 'JJ' : 'Adjetivo', 'VP' : 'Frase verbal',
          'JJR' : 'Adjetivo, comparativo', 'JJS' : 'Adjetivo, superlativo', 'LS' : 'Marcador de elemento de lista',
          'MD' : 'modal', 'NN' : 'Sustantivo, singular o masivo', 'NNS' : 'Sustantivo, plural', 'PP' : 'Frase de preposición',
          'NNP' : 'nombre propio, frase singular', 'NNPS' : 'nombre propio, plural', 'PDT' : 'Predeterminante',
          'POS' : 'final posesivo', 'PRP' : 'Frase de pronombre personal', 'PRP' : 'Frase de pronombre posesivo',
          'RB' : 'Adverbio', 'RBR' : 'Adverbio, comparativo', 'RBS' : 'Adverbio, superlativo', 'RP' : 'Partícula',
          'S' : 'Cláusula declarativa simple', 'SBAR' : 'Cláusula introducida por una conjunción subordinada (posiblemente vacía)',
          'SBARQ' : 'Pregunta directa introducida por una wh-Word o una frase wh.', 
          'SINV' : 'oración declarativa invertida, es decir, una en la que el sujeto sigue el verbo tenso o modal.',
          'SQ' : 'Pregunta sí/no invertida, o cláusula principal de una pregunta wh, siguiendo la frase wh en SBARQ.',
          'SYM' : 'Símbolo', 'VBD' : 'Verbo, tiempo pasado', 'VBG' : 'Verbo, gerundio o participio presente',
          'VBN' : 'Verbo, participio pasado', 'VBP' : 'Verbo, presente no singular de tercera persona',
          'VBZ' : 'Verbo, tercera persona singular presente', 'WDT' : 'determinante Wh', 'WP' : 'Wh-pronombre',
          'WP' : 'pronombre wh posesivo', 'WRB' : 'Adverbio Wh'}

In [42]:
histo = widgets.Dropdown(
    options=[elem[0] for elem in tags if(elem[1] not in ['TO','NN','IN','FN','FW','CC','DT','NNP','VB','VBD','VBG','VBN'])],
    value= lista[0],
    description='Evolución de la palabra clave:',
)
def buscar_tag(pareja):
    tag = [elem[1] for elem in tags if elem[0]==pareja]
    return tag[0]
tipo_tag = widgets.HTML(
    value="El tag de la palabra es <b> "+ str(buscar_tag(lista[0])) +"</b>: " + dict_tags[buscar_tag(lista[0])] ,
    placeholder='',
    description='',
)

valores = contar_palabra(tabla_dias, lista[0])
trace = px.line(x=valores.index,y =valores[lista[0]], line_shape='spline')
g = go.FigureWidget(data=trace)
def response(change):
    if histo.value in histo.options:
        df_temp = contar_palabra(tabla_dias, histo.value)
        tipo_tag.value = "El tag de la palabra es <b> "+ str(buscar_tag(histo.value)) +"</b>: " + dict_tags[buscar_tag(histo.value)]
        with g.batch_update():
            g.data[0].x = df_temp.index
            g.data[0].y = df_temp[histo.value]
            g.data[0].name = histo.value

                

histo.observe(response, names="value")

container = widgets.HBox([histo,tipo_tag])
widgets.VBox([container, g])